## Getting PBP

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import percentileofscore as perc
from simFootballPBP import *

'''
Functions
'''

def scoreDiff(row):
    if row['recTeam'] == row['awayTeam']:
        return row['homeScore'] - row['awayScore']
    else:
        return row['awayScore'] - row['homeScore']

def secSinceHalf(row):
    return 1800 - int(row['totTime'])

def fieldPosScore(row):
    if row['dist2goal'] >= 60:
        return 1.0
    elif row['dist2goal'] >= 50:
        return 1.1 ** (40 - row['dist2goal'])
    else:
        return 1.1**10 * 1.2**(50 - row['dist2goal'])

def distScore(row):
    if int(row['distance']) >= 10:
        return 0.2
    elif 7 <= row['distance'] <= 9:
        return 0.4
    elif 4 <= row['distance'] <= 6:
        return 0.6
    elif 2 <= row['distance'] <= 3:
        return 0.8
    else:
        return 1.0

def scoreDiffMult(pointdiff):
    if pointdiff > 0:
        return 1
    elif pointdiff == 0:
        return 2
    elif pointdiff < -8:
        return 3
    else:
        return 4

def timeMult(row):
    sec = secSinceHalf(row)
    PD = scoreDiff(row)
    
    if PD <= 0 and sec >= 0:
        return ((sec * 0.001)**3) + 1.
    else:
        return 1.

def surrenderIndex(row):
    pd = scoreDiff(row)
    return fieldPosScore(row) * distScore(row) * scoreDiffMult(pd) * timeMult(row)

def receiving(row):
    if row['teamPoss'] == row['homeTeam']:
        return row['awayTeam']
    else:
        return row['homeTeam']
    
def touchback(row):
    if 'Touchback' in row['play']:
        return "Touchback"
    elif 'No return' in row['play']:
        return "No return"
    else:
        return "Returned"

def puntDist(row):
    if 'BLOCKED' in row.play:
        return 0
    elif 'Touchback' in row.play:
        return row.dist2goal - 20
    else:
        return int(row.play.split('of ')[-1].split(' yards')[0])
    
def stringify(row):
    num = ['st','nd','rd','th']
    if row['awayTeam'] == row['teamPoss']:
        puntScore = row['awayScore']
        recScore = row['homeScore']
    else:
        puntScore = row['homeScore']
        recScore = row['awayScore']
    return "Q%s - %s - %i%s and %i\n%s %s - %s %s\n"%(row['Q'],row['time'],row['down'],num[int(row['down'])-1],row['distance'],row['teamPoss'],puntScore,row['recTeam'],recScore)


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
S = 25

sData = []
idList, idDict = getSeasonIDs(S)

print('Season:',S)
print('Games:',len(idList))

Season: 25
Games: 14


In [3]:
print('Gathering PBP...')
for i in idList:
    sData.append(getGameData(S,i,idDict))

sDF = pd.concat(sData)
sDF.to_csv('PBP/S%sPBP.csv'%S)

print('PBP for S%s saved!'%S)

Gathering PBP...
PBP for S25 saved!


In [5]:
print('Combining all Seasons...')
import os
allDF = pd.concat([pd.read_csv('PBP/%s'%p) for p in next(os.walk('PBP'))[2] if '.csv' in p])
allDF = allDF.reset_index().sort_values('S')
# allDF.to_csv('allPBP.csv')

print('Getting Punts...')
puntDF = allDF[allDF['play'].str.contains('Punt ')]

print('Done!')

Combining all Seasons...
Getting Punts...
Done!


In [6]:
print('Cleaning up PuntDF...')
NFL = np.load("2009-2018_surrender_indices.npy")
puntDF['recTeam'] = puntDF.apply(lambda row : receiving(row),axis=1)
puntDF['surrenderIndex'] = puntDF.apply(lambda row : surrenderIndex(row),axis=1)
puntDF = puntDF[['S','W','gameID','Q','time','awayTeam','awayScore','homeScore','homeTeam'
                 ,'teamPoss','recTeam','down','distance','side','dist2goal','surrenderIndex','play']]
puntDF = puntDF.sort_values('surrenderIndex',ascending=False)
puntDF['NFLpercentiles'] = puntDF.apply(lambda row : perc(NFL,row['surrenderIndex']),axis=1)
puntDF['percentiles'] = puntDF.apply(lambda row : perc(puntDF['surrenderIndex'],row['surrenderIndex']),axis=1)
puntDF['result'] = puntDF.apply(lambda row : touchback(row),axis = 1)
puntDF['scoreDiff'] = puntDF.apply(lambda row : scoreDiff(row),axis = 1)
puntDF['puntDist'] = puntDF.apply(lambda row : puntDist(row),axis = 1)
puntDF['puntEndLoc'] = puntDF['dist2goal'] - puntDF['puntDist']
puntDF['surrenderRank'] = puntDF.surrenderIndex.rank(method='max',ascending=False).astype('int')
puntDF['situation'] = puntDF.apply(lambda row : stringify(row), axis = 1)

puntDF = puntDF[['S', 'W', 'gameID', 'Q', 'time', 'awayTeam', 'awayScore', 'homeScore',
       'homeTeam', 'teamPoss', 'recTeam', 'down', 'distance', 'dist2goal', 'puntDist',
       'puntEndLoc', 'surrenderIndex','surrenderRank', 'percentiles', 'NFLpercentiles', 'play',
       'situation','result']]

print('Exporting Surrender DF...')
puntDF.to_csv('surrender.csv')

print('Done!')

Cleaning up PuntDF...


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Exporting Surrender DF...
Done!


## Bot Functions

In [7]:
s25 = pd.read_csv('PBP/S25PBP.csv')


In [13]:
s25[(s25.S == 25) & (s25.W == 1) & (s25.homeTeam == 'COL') | (s25.awayTeam == 'COL')]

Unnamed: 0  teamID  Q   time  totTime  down  distance side  yard  \
836            0       3  1  15:00     3600   NaN       NaN  COL  30.0   
837            1      11  1  15:00     3600   NaN       NaN  NaN   NaN   
838            2      11  1  15:00     3600   NaN       NaN  NaN   NaN   
839            3      11  1  15:00     3600   NaN       NaN  SAR  24.0   
840            4      11  1  14:53     3593   1.0      10.0  SAR  24.0   
...          ...     ... ..    ...      ...   ...       ...  ...   ...   
2519         191       3  4   1:37       97   4.0       5.0  CHI  45.0   
2520         192       9  4   1:11       71   1.0      10.0  CHI  45.0   
2521         193       9  4   0:54       54   2.0      10.0  CHI  45.0   
2522         194       9  4   0:27       27   3.0      10.0  CHI  45.0   
2523         195       3  4   0:00        0   4.0       8.0  CHI  47.0   

                                                   play  gameID   S  W  \
836                               Banana, S. kicks off.    6373  25  1   
837                    The kick sails into the endzone.    6373  25  1   
838           Clemente, C. takes it out of the endzone!    6373  25  1   
839   Kickoff of 83 yards. Returned by Clemente, C. ...    6373  25  1   
840   Rush by Chan, R. for a short gain. Tackle by R...    6373  25  1   
...                                                 ...     ...  .. ..   
2519                                 Turnover on downs.    6380  25  2   
2520  Rush by Yoda, B. for a short gain. Tackle by B...    6380  25  2   
2521  Rush by Yoda, B. for 0 yds. Tackle by Blacksto...    6380  25  2   
2522  Rush by Yoda, B. for 2 yds. Tackle by Murder-M...    6380  25  2   
2523       Punt by Sidekick, S. of 52 yards. Touchback.    6380  25  2   

     homeTeam awayTeam teamPoss  dist2goal  awayScore  homeScore  
836       COL      SAR      COL       70.0          0          0  
837       COL      SAR      SAR        NaN          0          0  
838       COL      SAR      SAR        NaN          0          0  
839       COL      SAR      SAR       76.0          0          0  
840       COL      SAR      SAR       76.0          0          0  
...       ...      ...      ...        ...        ...        ...  
2519      CHI      COL      COL       45.0          6         17  
2520      CHI      COL      CHI       55.0          6         17  
2521      CHI      COL      CHI       55.0          6         17  
2522      CHI      COL      CHI       55.0          6         17  
2523      CHI      COL      CHI       53.0          6         17  

[386 rows x 19 columns]